# Introduction

This notebook trains a linear discriminant analysis model on preprocessed EEG data to predict behavioral data from neural features.

An example of this can be found in the paper, "Human stereoEEG recordings reveal network dynamics of decision-making in a rule-switching task" by Wal et al.

Behavioral data and preprocessed neural data from Subject 06 in the NCSL EFRI datasetwill be used in this notebook.

# Setup

## Imports

In [1]:
import h5py 
import mat73
import numpy as np

## Load File Paths

In [9]:
# ncsl_share = '/run/user/1000/gvfs/smb-share:server=10.162.37.21,share=main'
# data_path = f'Data/Subject06_snapshot_normalized.npy'
subs = ['06']
file_paths = {}

for sub in subs:
    # create a dictionary holding the file paths
    ncsl_share = '/mnt/ncsl_share'
    file_paths[sub] = {
        'setup_path': ncsl_share + f'/Public/EFRI/1_formatted/SUBJECT{sub}/EFRI{sub}_WAR_SES1_Setup.mat',
        'raw_path': ncsl_share + f'/Public/EFRI/1_formatted/SUBJECT{sub}/EFRI{sub}_WAR_SES1_Raw.mat',
        'data_path': ncsl_share + f'/Daniel/Data/Trial_by_Chan_by_Freq_by_Time_Snapshots/Subject{sub}_snapshot_normalized.npy', # movement onset as event
        # 'data_path' : ncsl_share + f'/Daniel/Data/Trial_by_Chan_by_Freq_by_Time_Snapshots/show-card_pre-2sec_post-4sec/Subject{sub}_snapshot_normalized.npy', # visual cue as event
        'out_path_metrics': f'Metrics/Subject{sub}',
        'out_path_plots': f'Plots/Subject{sub}'
    }

In [10]:
raw_file = h5py.File(file_paths['06']['raw_path'])
setup_data = mat73.loadmat(file_paths['06']['setup_path'])

out_path_plots = file_paths['06']['out_path_plots']
out_path_metrics = file_paths['06']['out_path_metrics']

In [11]:
setup_data.keys()

dict_keys(['elec_area', 'elec_ind', 'elec_name', 'filters', 'trial_times', 'trial_words'])

## Instantiate variables

In [12]:
bets = setup_data['filters']['bets']

good_trials = np.where(np.isnan(bets) == False)[0] # extract indices of trials without the 'nan'

bets = bets[good_trials] # get the bet values for the good trials
subject_cards = setup_data['filters']['card1'][good_trials] # get the subject's card values for the good trials

In [13]:
elec_names = np.array(setup_data['elec_name'])
elec_areas = np.array(setup_data['elec_area'])

In [14]:
data = np.load(file_paths['06']['data_path'])
y = np.asarray([(0 if bet == 5 else 1) for bet in bets]) # 0 = low bet ($5), 1 = high bet ($20)

## Matplotlib Settings

In [15]:
import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18

## Create Frequency Bands

In [9]:
wavelet_freqs = np.logspace(np.log2(2),np.log2(150),num=63,base=2)

frequency_band_indices ={
    "Delta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 0.5 and freq < 4],
    "Theta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 4 and freq < 8],
    "Alpha" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 8 and freq < 14],
    "Beta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 14 and freq < 30],
    "Gamma" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 30]
}

In [10]:
f_band_data = np.zeros((data.shape[0], data.shape[1], 5, data.shape[3]))

for i, key in enumerate(frequency_band_indices):
    f_band_data[:,:,i,:] = data[:,:,frequency_band_indices[key],:].mean(2)

# Model Training

# Analysis

In [41]:
def find_shared_brain_areas(brain_area_combinations):
    flatten_list = lambda l: [item for sublist in l for item in sublist]
    unique_brain_areas = np.unique(flatten_list(brain_area_combinations))

    shared_brain_areas = []
    for brain_area in unique_brain_areas:
        i = 0
        for combination in brain_area_combinations:
            if brain_area in combination:
                i += 1
        if i > 1:
            shared_brain_areas.append([brain_area, i])

    shared_brain_areas.sort(key=lambda x: x[1], reverse=True)
    
    return shared_brain_areas

### Find which brain regions that contribute to decision-making are shared among subjects (movement onset)

In [39]:
channel_combinations_for_movement = []
optimal_channel_combinations_for_movement = []

subs = ['06','07','10','12','13','15','16','17','18','21']

for sub in subs:
    channel_combinations_for_movement.append(np.load(f'Metrics/Subject{sub}_optimal_time_window_channel_combination.npy'))
    optimal_channel_combinations_for_movement.append(np.load(f'Metrics/Subject{sub}_optimal_time_window_optimal_channel_combination.npy'))

In [42]:
find_shared_brain_areas(optimal_channel_combinations_for_movement)

[['angular gyrus R', 3],
 ['fusiform gyrus L', 3],
 ['middle temporal gyrus L', 3],
 ['occipital gyrus L', 3],
 ['cingulate cortex (posterior) R', 2],
 ['inferior temporal gyrus L', 2],
 ['insular cortex (anterior) R', 2],
 ['middle temporal gyrus R', 2],
 ['parietal operculum R', 2],
 ['parietooccipital sulcus L', 2],
 ['superior temporal gyrus (planum temporale) L', 2],
 ['superior temporal gyrus (planum temporale) R', 2],
 ['superior temporal sulcus R', 2],
 ['supramarginal gyrus L', 2]]

In [43]:
find_shared_brain_areas(channel_combinations_for_movement)

[['middle temporal gyrus L', 5],
 ['angular gyrus R', 4],
 ['fusiform gyrus L', 4],
 ['cingulate cortex (posterior) R', 3],
 ['inferior frontal gyrus (pars orbitalis) R', 3],
 ['inferior temporal sulcus R', 3],
 ['middle temporal gyrus R', 3],
 ['superior temporal gyrus (planum temporale) R', 3],
 ['cuneus R', 2],
 ['fusiform gyrus R', 2],
 ['hippocampus (anterior) L', 2],
 ['hippocampus (anterior) R', 2],
 ['hippocampus (posterior) R', 2],
 ['inferior frontal gyrus (pars opercularis) R', 2],
 ['inferior temporal gyrus L', 2],
 ['inferior temporal gyrus R', 2],
 ['insular cortex (anterior) R', 2],
 ['insular cortex (posterior) R', 2],
 ['intraparietal sulcus R', 2],
 ['occipital gyrus L', 2],
 ['occipital gyrus R', 2],
 ['parietal operculum R', 2],
 ['parietooccipital sulcus L', 2],
 ['superior temporal gyrus (planum temporale) L', 2],
 ['superior temporal sulcus R', 2],
 ['supramarginal gyrus L', 2],
 ['supramarginal gyrus R', 2]]

### Find which brain regions that contribute to decision-making are shared among subjects (visual cue)

In [44]:
channel_combinations_for_vis_stim = []
optimal_channel_combinations_for_vis_stim = []

subs = ['06','07','10','12','13','15','16','17','18','21']

for sub in subs:
    channel_combinations_for_vis_stim.append(np.load(f'Metrics/Subject{sub}_vis_stim_optimal_time_window_channel_combination.npy'))
    optimal_channel_combinations_for_vis_stim.append(np.load(f'Metrics/Subject{sub}_vis_stim_optimal_time_window_optimal_channel_combination.npy'))

In [45]:
find_shared_brain_areas(optimal_channel_combinations_for_vis_stim)

[['cingulate cortex (posterior) R', 3],
 ['fusiform gyrus L', 3],
 ['middle temporal gyrus R', 3],
 ['superior temporal gyrus (planum temporale) L', 3],
 ['angular gyrus R', 2],
 ['cuneus R', 2],
 ['fusiform gyrus R', 2],
 ['hippocampus (posterior) L', 2],
 ['insular cortex (posterior) R', 2],
 ['intraparietal sulcus R', 2],
 ['middle temporal gyrus L', 2],
 ['occipital gyrus (lateral) L', 2],
 ['occipital gyrus L', 2],
 ['parietal operculum L', 2],
 ['parietooccipital sulcus L', 2],
 ['superior temporal gyrus (planum temporale) R', 2],
 ['supramarginal gyrus L', 2],
 ['temporal pole R', 2]]

In [46]:
find_shared_brain_areas(channel_combinations_for_vis_stim)

[['cingulate cortex (posterior) R', 4],
 ['middle temporal gyrus R', 4],
 ['fusiform gyrus L', 3],
 ['parietal operculum L', 3],
 ['precuneus R', 3],
 ['superior temporal gyrus (planum temporale) R', 3],
 ['supramarginal gyrus L', 3],
 ['angular gyrus R', 2],
 ['cuneus R', 2],
 ['entorhinal cortex L', 2],
 ['fusiform gyrus R', 2],
 ['hippocampus (anterior) L', 2],
 ['hippocampus (anterior) R', 2],
 ['hippocampus (posterior) R', 2],
 ['inferior temporal sulcus R', 2],
 ['insular cortex (posterior) R', 2],
 ['intraparietal sulcus R', 2],
 ['lingula gyrus R', 2],
 ['middle temporal gyrus L', 2],
 ['occipital gyrus L', 2],
 ['occipital gyrus R', 2],
 ['superior temporal gyrus (planum temporale) L', 2],
 ['superior temporal sulcus R', 2],
 ['temporal pole R', 2]]